In [28]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron
from sklearn import model_selection
import matplotlib.pyplot as plt
df = pd.read_csv("diabetes.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype
---  ------                 --------------   -----
 0   Diabetes               253680 non-null  int64
 1   HighBP                 253680 non-null  int64
 2   HighChol               253680 non-null  int64
 3   BMI                    253680 non-null  int64
 4   Smoker                 253680 non-null  int64
 5   Stroke                 253680 non-null  int64
 6   Myocardial             253680 non-null  int64
 7   PhysActivity           253680 non-null  int64
 8   Fruit                  253680 non-null  int64
 9   Vegetables             253680 non-null  int64
 10  HeavyDrinker           253680 non-null  int64
 11  HasHealthcare          253680 non-null  int64
 12  NotAbleToAffordDoctor  253680 non-null  int64
 13  GeneralHealth          253680 non-null  int64
 14  MentalHealth           253680 non-null  int64
 15  PhysicalHealth   

In [29]:
df.head()

,Diabetes,HighBP,HighChol,BMI,Smoker,Stroke,Myocardial,PhysActivity,Fruit,Vegetables,...,NotAbleToAffordDoctor,GeneralHealth,MentalHealth,PhysicalHealth,HardToClimbStairs,BiologicalSex,AgeBracket,EducationBracket,IncomeBracket,Zodiac
0,0,1,1,40,1,0,0,0,0,1,...,0,5,18,15,1,1,9,4,3,10
1,0,0,0,25,1,0,0,1,0,0,...,1,3,0,0,0,1,7,6,1,11
2,0,1,1,28,0,0,0,0,1,0,...,1,5,30,30,1,1,9,4,8,2
3,0,1,0,27,0,0,0,1,1,1,...,0,2,0,0,0,1,11,3,6,11
4,0,1,1,24,0,0,0,1,1,1,...,0,2,3,0,0,1,11,5,4,8


1


In [30]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn import metrics

def myPerceptron(X, y, random_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
    clf = Perceptron(tol=1e-3, random_state=0, class_weight = 'balanced')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    auc_roc = metrics.roc_auc_score(y_test, y_pred)
    print('*AUC is ', auc_roc)
    print('')
    return clf


In [31]:
predictors = ['HighBP', 'HighChol', 'BMI', 'Smoker','Stroke','Myocardial','PhysActivity','Fruit','Vegetables','HeavyDrinker','HasHealthcare','NotAbleToAffordDoctor','GeneralHealth','MentalHealth','PhysicalHealth','HardToClimbStairs', 'BiologicalSex', 'AgeBracket', 'EducationBracket', 'IncomeBracket', 'Zodiac']
X = df[predictors]
y = df['Diabetes']
scaled = StandardScaler().fit_transform(X)
scaled = pd.DataFrame(scaled, columns=X.columns)

In [20]:
model = myPerceptron(scaled, y, 0)

*AUC is  0.622603402033001



Build and train a Perceptron (one input layer, one output layer, no hidden layers and no activation functions) to classify diabetes from the rest of the dataset. What is the AUC of this model?

I built a Perceptron function that takes in three arguments: predictors, target and random state. I split data into training and testing sets and fitted the perceptron with balanced class_weight. Finally, I computed AUC.


I set 1e-3 for the tolerance for the stopping criterion. If the change in the error function between iterations is less than tol, then the algorithm will terminate. The balanced option will automatically adjust the weights based on the number of samples in each class.

The result shows that the AUC is 0.622603402033001, which is not very high comparing the result from last homework

This result indicates that in order to obtain a better AUC for the perceptron algorithm on the given data, we may need to modify or fine-tune the algorithm's parameters.





2

In [21]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import torch.optim as optim



# Define the neural network
class DiabetesClassifier1(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, number_of_hidden,activation):
        super(DiabetesClassifier1, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.hidden_layer = nn.ModuleList()
        for i in range (number_of_hidden):
            self.hidden_layer.append(nn.Linear(hidden_dim, hidden_dim))
        self.output_layer = nn.Linear(hidden_dim, output_dim)

        if activation == 'relu':
            self.activation1 = nn.ReLU()
        elif activation == 'sigmod':
            self.activation1 = nn.Sigmoid()
        else:
            self.activation1 = nn.Identity()


        
    def forward(self, x):
        x = self.activation1(self.input_layer(x))
        for l in self.hidden_layer:
            x = self.activation1(l(x))
        x = self.output_layer(x)
        return x
    

def train(X_train1, X_test1, y_train1, y_test1,number_of_hidden,activation):
    input_dim = X_train1.shape[1]
    hidden_dim = 10
    output_dim = 1
    learning_rate = 0.05

    num_epochs = 100

    model = DiabetesClassifier1(input_dim, hidden_dim, output_dim, number_of_hidden, activation)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(torch.from_numpy(X_train1.values).float())
        loss = criterion(outputs, torch.from_numpy(y_train1.values).float().view(-1,1))
        loss.backward()

        optimizer.step()
    with torch.no_grad():
        auc = roc_auc_score(y_test1.values, torch.sigmoid(model(torch.from_numpy(X_test1.values).float())).numpy().flatten()) # Convert Pandas Series to numpy arrays
    return auc



In [23]:
functions = ['relu', 'sigmoid', 'linear']
num = [1,2]
X_train, X_test, y_train, y_test = train_test_split(scaled, y, test_size=0.3, random_state=0)
for n in num:
    for f in functions:
        value = train(X_train, X_test, y_train, y_test, n, f)
        print(f" n={n}, f={f}, auc={value:.4f}")

 n=1, f=relu, auc=0.8299
 n=1, f=sigmoid, auc=0.8235
 n=1, f=linear, auc=0.8235
 n=2, f=relu, auc=0.8294
 n=2, f=sigmoid, auc=0.8235
 n=2, f=linear, auc=0.8235


Build and train a feedforward neural network with at least one hidden layer to classify diabetes from the rest of the dataset. Make sure to try different numbers of hidden layers and different activation functions (at a minimum reLU and sigmoid). Doing so: How does AUC vary as a function of the number of hidden layers and is it dependent on the kind of activation function used (make sure to include “no activation function” in your comparison). How does this network perform relative to the Perceptron?

I built a neural network model, and I tried different number of layers, different algorithm and I computed the value of AUC for each case.

BCEWithLogitsLoss was used for criterion to combine the sigmoid activation and binary cross-entropy loss into a single function.
Also the number of hidden units and other variable were set

The result shows that auc are around 0.8, the number of layers did not show too much difference. In terms of activation function, Relu shows better performance.

Based on the results obtained, we can conclude that the number of hidden layers in the neural network does not have a significant impact on the AUC value, while the choice of activation function has only a minor effect.

3


In [26]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import roc_auc_score

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.cnn1 = nn.Conv1d(1, 32, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64*5, 128)
        self.fc2 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = self.cnn1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.cnn2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64*5)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

In [27]:

X_train, X_test, y_train, y_test = train_test_split(scaled.values, y.values, test_size=0.3, random_state=0)
xr, yr = torch.from_numpy(X_train.astype(np.float32)).unsqueeze(1), torch.from_numpy(y_train.astype(np.int64))
xe, ye = torch.from_numpy(X_test.astype(np.float32)).unsqueeze(1), torch.from_numpy(y_test.astype(np.int64))

num_epochs = 100
learning_rate = 0.01
model = CNN()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    loss = criterion(model(xr),yr)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, loss.item()))
with torch.no_grad():
    outputs = model(xe)
    pred = torch.max(outputs.data,1)[1]
    y_scores = nn.functional.softmax(outputs,dim =1 )[:,1].numpy()
    auc = roc_auc_score(ye.numpy(), y_scores)
print('AUC: ', auc)

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


Epoch [10/100], Loss: 0.4138
Epoch [20/100], Loss: 0.3464
Epoch [30/100], Loss: 0.3319
Epoch [40/100], Loss: 0.3250
Epoch [50/100], Loss: 0.3210
Epoch [60/100], Loss: 0.3186
Epoch [70/100], Loss: 0.3168
Epoch [80/100], Loss: 0.3155
Epoch [90/100], Loss: 0.3147
Epoch [100/100], Loss: 0.3141
AUC:  0.8280848221597302


Build and train a “deep” network (at least 2 hidden layers) to classify diabetes from the rest of the dataset. Given the nature of this dataset, is there a benefit of using a CNN or RNN for the classification?

I built a CNN model, which consists of two convolutional layers with ReLU activation, each followed by a max-pooling layer. This network can be used to classify 1D input sequences into two classes.

CrossEntropyLoss were used for criterion because it computes the cross-entropy loss between the predicted class probabilities and the ground truth labels. Also the loss was computed in each epoch to minimize the loss function.

the AUC is 0.828 which is a little better than the result from last question

The result shows that the auc is little higher than neural network with one or two hidden layers. However, I do not think it is worth to use CNN or RNN. CNN is often used for data that has a spatial or temporal structure and RNN is often used for data that has a sequential or time-dependent structure. Both CNN and RNN are computationally expensive and can be slow to train on large datasets.

4

In [32]:
predictors1 = ['Diabetes','HighBP', 'HighChol', 'Smoker','Stroke','Myocardial','PhysActivity','Fruit','Vegetables','HeavyDrinker','HasHealthcare','NotAbleToAffordDoctor','GeneralHealth','MentalHealth','PhysicalHealth','HardToClimbStairs', 'BiologicalSex', 'AgeBracket', 'EducationBracket', 'IncomeBracket', 'Zodiac']
X1 = df[predictors1]
y1 = df['BMI']
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
y1 = scaler.fit_transform(y1.values.reshape(-1, 1))
scaled1 = StandardScaler().fit_transform(X1)
scaled1 = pd.DataFrame(scaled1, columns=X1.columns)

In [41]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor


def BMIPredictor(scaled1,y1,num, activation, lr):
    X_train1, X_test1, y_train1, y_test1 = train_test_split(scaled1, y1, test_size=0.3, random_state=0)
    mlp = MLPRegressor(hidden_layer_sizes=num, activation=activation, solver= 'adam', max_iter=200, learning_rate_init=lr)
    mlp.fit(X_train1, y_train1)
    y_pred = mlp.predict(X_test1)
    return mean_squared_error(y_test1, y_pred, squared=False)
functions = ['relu', 'identity', 'tanh','logistic']
for i in functions:
    print(BMIPredictor(scaled1, y1,100, i, 0.1))



/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.07659163137150962


/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.35365862722637986


/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.10531186257280778


/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.07499549948581513


Build and train a feedforward neural network with one hidden layer to predict BMI from the rest of the dataset. Use RMSE to assess the accuracy of your model. Does the RMSE depend on the activation function used?


I changed the target from Diabetes to BMI. I did the same way as before. Since theoretically BMI can go from 0 to infinity so I normalize it to 0 to 1


MLPRegressor were used to predictor BMI. I set adam for solver and the solver will run for a maximum of 200 iterations. Other parameters can be set by passing in

The result for relu and logistic are about 0.07, for identity is 0.35, for tanh is 0.1

According to the findings, the activation functions "relu" and "logistic" have the lowest RMSE (root mean square error), which suggests they perform better than "identity" and "tanh" whose RMSE is slightly higher. The reason is probably identity returns the same value as the input without any transformation and Tanh maps the input to a range between -1 and 1 which do not fit with the dataset

5

In [34]:
hidden_layer = [100, (100,100), (100, 100, 100)]
learning_rate = [0.05, 0.1]

for n in hidden_layer:
    for l in learning_rate:
        rmse = BMIPredictor(scaled1, y1,n, 'logistic', l)
        print(f"hidden={n}, learn_rate={l}, RMSE={rmse:.4f}")


/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


hidden=100, learn_rate=0.05, RMSE=0.0710


/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


hidden=100, learn_rate=0.1, RMSE=0.0767


/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

hidden=(100, 100), learn_rate=0.1, RMSE=0.0708


/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


hidden=(100, 100, 100), learn_rate=0.05, RMSE=0.0709


/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


hidden=(100, 100, 100), learn_rate=0.1, RMSE=0.0769


Build and train a neural network of your choice to predict BMI from the rest of your dataset. How low can you get RMSE and what design choices does RMSE seem to depend on?

I used the same function built from last question, but this time I changed the number of layers and the learning rate, then I computed the value of RMSE to assess the accuracy of the model

I did not include the activation function as one of the variable is that logistic and relu shows the same result and the other two do not fit with the data.

The values of RMSE are all around 0.07

Based on the results, it can be inferred that the accuracy of the model is not significantly affected by the number of layers or the learning rate used. However, the choice of activation function has a substantial impact on the accuracy of the model, as seen in the previous question.

6

In [35]:
corr_matrix = df.corr()
mask = (abs(corr_matrix) >= 0.4) & (corr_matrix != 1)

filtered_corr = corr_matrix.where(mask).stack()


print(filtered_corr)

GeneralHealth      PhysicalHealth       0.524364
                   HardToClimbStairs    0.456920
PhysicalHealth     GeneralHealth        0.524364
                   HardToClimbStairs    0.478417
HardToClimbStairs  GeneralHealth        0.456920
                   PhysicalHealth       0.478417
EducationBracket   IncomeBracket        0.449106
IncomeBracket      EducationBracket     0.449106
dtype: float64


In [42]:
df.head(5)

,Diabetes,HighBP,HighChol,BMI,Smoker,Stroke,Myocardial,PhysActivity,Fruit,Vegetables,...,NotAbleToAffordDoctor,GeneralHealth,MentalHealth,PhysicalHealth,HardToClimbStairs,BiologicalSex,AgeBracket,EducationBracket,IncomeBracket,Zodiac
0,0,1,1,40,1,0,0,0,0,1,...,0,5,18,15,1,1,9,4,3,10
1,0,0,0,25,1,0,0,1,0,0,...,1,3,0,0,0,1,7,6,1,11
2,0,1,1,28,0,0,0,0,1,0,...,1,5,30,30,1,1,9,4,8,2
3,0,1,0,27,0,0,0,1,1,1,...,0,2,0,0,0,1,11,3,6,11
4,0,1,1,24,0,0,0,1,1,1,...,0,2,3,0,0,1,11,5,4,8


In [36]:
columns_to_drop = ['IncomeBracket','HardToClimbStairs','PhysActivity', 'GeneralHealth', 'PhysicalHealth','MentalHealth']
df1 = df.drop(columns=columns_to_drop)

In [37]:
predictors = list(df1.columns)
predictors.remove('Diabetes')
predictors1 = list(df1.columns)
predictors1.remove('BMI')
X = df1[predictors]
y = df1['Diabetes']
X1 = df1[predictors1]
y1 = df1['BMI']
scaled = StandardScaler().fit_transform(X)
scaled = pd.DataFrame(scaled, columns=X.columns)
scaled1 = StandardScaler().fit_transform(X1)
scaled1 = pd.DataFrame(scaled1, columns=X1.columns)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.3, random_state=0)

In [38]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
def MLPPredictor(scaled1,y1,name,num, activation, lr):
    X_train1, X_test1, y_train1, y_test1 = train_test_split(scaled1, y1, test_size=0.3, random_state=0)
    mlp = MLPRegressor(hidden_layer_sizes=num, activation=activation, solver= 'adam', max_iter=200, learning_rate_init=lr)
    mlp.fit(X_train1, y_train1)
    if name != '':
        print(f'After dropping {name}:')
    y_pred = mlp.predict(X_test1)
    print('rmse: ',mean_squared_error(y_test1, y_pred, squared=False))

In [12]:

model1 = MLPPredictor(scaled, y, '',100,'relu',0.1)
for i in range(len(predictors)):
    temp = predictors[:]
    temp.pop(i)
    model2 = MLPPredictor(df[temp], y,predictors[i],100,'relu',0.1)

rmse:  0.3300032573685665
After dropping HighBP:
rmse:  0.3498361827060645
After dropping HighChol:
rmse:  0.34975850466481045
After dropping BMI:
rmse:  0.3486268465244555
After dropping Smoker:
rmse:  0.34852056844209717
After dropping Stroke:
rmse:  0.349803570414535
After dropping Myocardial:
rmse:  0.350209224741946
After dropping Fruit:
rmse:  0.3244663575707566
After dropping Vegetables:
rmse:  0.3485574770714167
After dropping HeavyDrinker:
rmse:  0.34846398459788436
After dropping HasHealthcare:
rmse:  0.34851372418996196
After dropping NotAbleToAffordDoctor:
rmse:  0.3541547063885861
After dropping BiologicalSex:
rmse:  0.34838042763411525
After dropping AgeBracket:
rmse:  0.3484328917553216
After dropping EducationBracket:
rmse:  0.34967164459289735
After dropping Zodiac:
rmse:  0.3483818577087824


In [39]:
from sklearn.neural_network import MLPRegressor
y1 = scaler.fit_transform(y1.values.reshape(-1, 1))
model1 = MLPPredictor(scaled1, y1, '',100,'relu',0.1)
for i in range(len(predictors1)):
    temp = predictors1[:]
    temp.pop(i)
    model2 = MLPPredictor(df[temp], y1,predictors1[i],100,'relu',0.1)

/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


rmse:  0.07801928794377691


/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


After dropping Diabetes:
rmse:  0.07656463805586343


/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

After dropping AgeBracket:
rmse:  0.07697459466728392


/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


After dropping EducationBracket:
rmse:  0.07919968841294536


/Users/yanggezheng/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


After dropping Zodiac:
rmse:  0.07737083805752383


Are there any predictors/features that have effectively no impact on the accuracy of these models? If so, please list them and comment briefly on your findings

I first looked at the relationship between each data item and dropped some highly correlated columns such as GeneralHealth and MentalHealth, and I also dropped some self-tested columns such as Hard to climb stairs.

As people have unique perspectives and experiences, their assessment of their own health can vary. For instance, one individual in the dataset might have risk factors such as high blood pressure, high cholesterol, smoking, physical inactivity, fruit deficiency, and high BMI, yet still rate their overall health as "very healthy" on a 5-point scale.

To evaluate the impact of each predictor, I ran the analysis with all predictors and then removed one predictor at a time to observe the changes. It was observed that BMI and diabetes were significant predictors in both scenarios. However, the predictors Zodiac, HasHealthcare, and Vegetables showed minimal effects on the outcome.

7

Neural networks have several advantages over traditional machine learning models such as logistic regression, random forests, decision trees, and adaboost_classification. 

For example, they are better suited to handle complex and nonlinear relationships between features, and can learn from large and unstructured data, while also generalizing well to new data. However, training neural networks can be computationally expensive and require more data and resources than other models.(mentioned in question 3) In contrast, logistic regression, random forests, decision trees, and adaboost_classification require fewer computational resources, but may not perform as well as neural networks when dealing with complex data. 

In this homework, the data were not too big complex nor huge. I can not really tell which one is better. The values of AUC and RMSE are pretty close. Ultimately, the choice of model depends on the specific problem and available data.